In [1]:
import gensim
from numpy import linalg
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath

!pip install ipython-autotime
%load_ext autotime

You should consider upgrading via the '/Users/stjepankusenic/POLAR_WEBE/venv/bin/python -m pip install --upgrade pip' command.
time: 991 µs (started: 2022-03-24 10:52:10 +01:00)


In [6]:
model_glove = glove2word2vec('/Users/stjepankusenic/POLAR_WEBE/data/raw/glove.42B.300d.txt','gensim_glove_300d.txt') ## needed for glove embeddings only 


time: 33.1 s (started: 2022-03-22 11:05:48 +01:00)


In [7]:
model_glove = gensim.models.KeyedVectors.load_word2vec_format("gensim_glove_300d.txt", binary=False)

time: 4min 40s (started: 2022-03-22 11:10:20 +01:00)


In [8]:
def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    temp_file.close()

time: 20.7 ms (started: 2022-03-22 15:32:24 +01:00)


In [9]:
generate_norm_embedding(model_glove,'glove_norm_300.mod')


/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_98626/4225384672.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for each_word in tqdm(model.vocab):


  0%|          | 0/1917494 [00:00<?, ?it/s]

time: 1min 5s (started: 2022-03-22 15:32:27 +01:00)


In [10]:
model_gn = gensim.models.KeyedVectors.load_word2vec_format('glove_norm_300.mod',binary=True) # ss


time: 25.3 s (started: 2022-03-22 15:33:52 +01:00)


In [11]:
current_model = model_gn 


time: 328 µs (started: 2022-03-22 15:34:19 +01:00)


In [ ]:
#%cd POLAR

In [12]:

list_antonym = []

with open('/Users/stjepankusenic/POLAR_WEBE/data/raw/Antonym_sets/LenciBenotto.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))


with open('/Users/stjepankusenic/POLAR_WEBE/data/raw/Antonym_sets/LenciBenotto.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
with open('/Users/stjepankusenic/POLAR_WEBE/data/raw/Antonym_sets/EVALution.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))

with open('/Users/stjepankusenic/POLAR_WEBE/data/raw/Antonym_sets/EVALution.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
                
list_antonym = list(dict.fromkeys(list_antonym).keys())

time: 40.2 ms (started: 2022-03-22 15:34:24 +01:00)


In [14]:
similarity_matrix = defaultdict(list)
for each_pair in tqdm(list_antonym):
    word1 = each_pair[0]
    word2 = each_pair[1]
    if word1 < word2:
        similarity_matrix[word1].append(word2)
    else:
        similarity_matrix[word2].append(word1)
    
all_similarity = defaultdict(dict)
for each_key in tqdm(similarity_matrix):
    for each_value in similarity_matrix[each_key]:
#         cosine_similarity([current_model[each_key]]
        all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
    
final_antonym_list = []
for index_counter, each_key in enumerate(tqdm(all_similarity)):
#     print(each_key,all_similarity[each_key])
    listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
#     print(listofTuples)
    final_antonym_list.append((each_key, listofTuples[0][0]))
print(len(final_antonym_list))

list_antonym = final_antonym_list

/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_98626/2902666931.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for each_pair in tqdm(list_antonym):


  0%|          | 0/3110 [00:00<?, ?it/s]

/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_98626/2902666931.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for each_key in tqdm(similarity_matrix):


  0%|          | 0/1468 [00:00<?, ?it/s]

/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_98626/2902666931.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index_counter, each_key in enumerate(tqdm(all_similarity)):


  0%|          | 0/1468 [00:00<?, ?it/s]

1468
time: 1.86 s (started: 2022-03-22 15:44:27 +01:00)


In [15]:
num_antonym = 1468


time: 407 µs (started: 2022-03-22 15:44:32 +01:00)


In [16]:
## Find the antonym difference vectors
antonymy_vector = []
for each_word_pair in list_antonym:
    antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
antonymy_vector = np.array(antonymy_vector)
print(antonymy_vector.shape)

(1468, 300)
time: 27.9 ms (started: 2022-03-22 15:44:37 +01:00)


In [17]:
import random

from scipy.spatial.distance import cosine as scipy_cosine
random.seed(42)

t1 = np.array(antonymy_vector)
dimension_similarity_matrix = defaultdict(dict)
for index_1, each_dim1 in enumerate(tqdm(t1)):
    for index_2, each_dim2 in enumerate(t1):
        dimension_similarity_matrix[index_1][index_2] = abs(1-scipy_cosine(each_dim1, each_dim2))
        
        
def get_set_score(final_list, each_dim):
    final_output = 0.0
    for each_vec in final_list:
        final_output += dimension_similarity_matrix[each_vec][each_dim]
    return final_output/(len(final_list))
        
def select_subset_dimension(dim_vector, num_dim):
    working_list = np.array(dim_vector)
    
    working_position_index = [i for i in range(working_list.shape[0])]
    final_position_index = []
    

    print('working list is ready, shape', working_list.shape)
    sel_dim = random.randrange(0, working_list.shape[0])

    final_position_index.append(sel_dim)
    
    working_position_index.remove(sel_dim)

    for test_count in tqdm(range(num_dim-1)):
        min_dim = None
        min_score = 1000
        for temp_index, each_dim in enumerate(working_position_index):
#             print(each_dim)
            temp_score = get_set_score(final_position_index, each_dim)
            if temp_score< min_score:
                min_score= temp_score
                min_dim = each_dim
        print(test_count,min_dim)
        final_position_index.append(min_dim)
        working_position_index.remove(min_dim)
#         print(working_list.shape, len(final_list))
    return final_position_index

/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_98626/1941183735.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index_1, each_dim1 in enumerate(tqdm(t1)):


  0%|          | 0/1468 [00:00<?, ?it/s]

time: 1min 46s (started: 2022-03-22 15:44:43 +01:00)


In [18]:
# use random dimension selection

random_antonym_vector = [i for i in range(len(antonymy_vector))]
random.shuffle(random_antonym_vector)
print(len(random_antonym_vector))

1468
time: 5.13 ms (started: 2022-03-22 15:46:31 +01:00)


In [19]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):
    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    print('New model size is',len(current_model.vocab), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model.vocab))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model.vocab))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model.vocab):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)

        
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()

time: 1.93 ms (started: 2022-03-22 15:46:34 +01:00)


In [20]:
def standard_normal_dist_model(model, new_filename):
    embedding_matrix = []
    embedding_vocab = []

    temp_file = open(new_filename,'wb')
    temp_file.write(str.encode(str(model.vectors.shape[0])+' '+str(model.vectors.shape[1])+'\n'))
    
    for each_word in tqdm(model.vocab):
        embedding_matrix.append(model[each_word])
        embedding_vocab.append(each_word)
    
    embedding_matrix = np.array(embedding_matrix)
    
    print('The shape of embedding matrix is {}'.format(embedding_matrix.shape))
    
    norm_embedding_matrix = (embedding_matrix - embedding_matrix.mean(0))/ embedding_matrix.std(0)
    
    for word_counter, each_word in enumerate(tqdm(embedding_vocab)):
#         assert each_word==embedding_vocab[word_counter],'Not matching!!!'
        
        temp_file.write(str.encode(each_word+' '))
        new_vector = norm_embedding_matrix[word_counter]
        temp_file.write(new_vector)
        temp_file.write(str.encode('\n'))
        
    del embedding_matrix
    del embedding_vocab
    temp_file.close()

time: 1.66 ms (started: 2022-03-22 15:46:39 +01:00)


In [21]:
def generate_embedding_path(current_model, embedding_path, binary, antonym_vector, curr_dim):
    curr_antonym_vector = antonymy_vector[antonym_vector[:curr_dim]]
    curr_antonymy_vector_inverse = np.linalg.pinv(np.transpose(curr_antonym_vector))
    new_embedding_dict = transform_to_antonym_space(current_model, embedding_path, binary,curr_antonymy_vector_inverse)

    return new_embedding_dict

time: 745 µs (started: 2022-03-22 15:46:46 +01:00)


In [22]:
import subprocess
dim_size = 500 # Number of POLAR dimenions
antonym_vector_method = random_antonym_vector # orthogonal_antonymy_vector, variance_antonymy_vector

time: 1.76 ms (started: 2022-03-22 15:46:50 +01:00)


In [26]:
import pandas as pd

time: 13.7 ms (started: 2022-03-22 15:57:39 +01:00)


In [27]:
company = pd.read_csv('/Users/stjepankusenic/POLAR_WEBE/data/interim/glove-ticker-fortune1000-us.csv')
ticker_list = company['Ticker'].str.lower()


time: 120 ms (started: 2022-03-22 15:57:40 +01:00)


In [28]:
ticker_word_embedding = dict()
for ticker in ticker_list:
  ticker_word_embedding[ticker] = current_model[ticker]

time: 591 ms (started: 2022-03-22 15:57:42 +01:00)


In [29]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):

    temp_dict = dict()

    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    print('New model size is',len(current_model), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)
        temp_dict[each_word] = new_vector
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()
    return temp_dict

time: 3.45 ms (started: 2022-03-22 15:57:48 +01:00)


In [30]:
ticker_new_embedding = generate_embedding_path(ticker_word_embedding, 'test_run',True ,random_antonym_vector,500)

New model size is 892 500


/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_98626/2921753199.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for each_word in tqdm(current_model):


  0%|          | 0/892 [00:00<?, ?it/s]

time: 407 ms (started: 2022-03-22 15:57:55 +01:00)


In [31]:
list_antonym[random_antonym_vector[0]]

('convict', 'pardon')

time: 51.3 ms (started: 2022-03-22 15:58:08 +01:00)


In [32]:
ticker_new_embedding.keys()

dict_keys(['wmt', 'amzn', 'xom', 'aapl', 'cvs', 'brka', 'unh', 'mck', 't', 'abc', 'googl', 'f', 'ci', 'cost', 'cvx', 'cah', 'jpm', 'gm', 'wba', 'vz', 'msft', 'mpc', 'kr', 'fnma', 'bac', 'hd', 'psx', 'cmcsa', 'antm', 'wfc', 'c', 'vlo', 'ge', 'dell', 'jnj', 'tgt', 'ibm', 'rtx', 'ba', 'fmcc', 'cnc', 'ups', 'low', 'intc', 'fb', 'fdx', 'met', 'dis', 'pg', 'pep', 'hum', 'pru', 'adm', 'aci', 'syy', 'lmt', 'hpq', 'et', 'gs', 'ms', 'cat', 'csco', 'pfe', 'hca', 'aig', 'axp', 'dal', 'mrk', 'aal', 'chtr', 'all', 'bby', 'ual', 'dow', 'tsn', 'tjx', 'orcl', 'gd', 'de', 'nke', 'pgr', 'ko', 'tecd', 'int', 'hon', 'cop', 'exc', 'noc', 'cof', 'pagp', 'abbv', 'epd', 'mmm', 'abt', 'trv', 'pm', 'hpe', 'arw', 'viac', 'dg', 'usb', 'sbux', 'bmy', 'mdlz', 'pcar', 'tmo', 'm', 'jbl', 'khc', 'duk', 'tsla', 'pbf', 'qcom', 'ngl', 'cbre', 'bkr', 'snx', 'dltr', 'cmi', 'unfi', 'mu', 'amgn', 'pag', 'v', 'avgo', 'nue', 'gild', 'luv', 'hal', 'lumn', 'ip', 'lly', 'afl', 'len', 'oxy', 'unp', 'rad', 'pnc', 'dd', 'so', 'an', '

time: 4.63 ms (started: 2022-03-22 15:58:10 +01:00)


In [33]:
ticker_new_embedding['amzn'][0]

0.023439674

time: 6.81 ms (started: 2022-03-22 15:58:14 +01:00)


In [34]:
antonym_500 = [list_antonym[x] for x in random_antonym_vector[:500]]

time: 5.14 ms (started: 2022-03-22 15:58:16 +01:00)


In [35]:
antonym_500[0]

('convict', 'pardon')

time: 3.99 ms (started: 2022-03-22 15:58:18 +01:00)


In [36]:
def make_polar_dict(company_name, antonym, embedding, top_n = False, n = 10):
  temp_dict = dict()
  temp_polar = embedding[company_name]

  if top_n:
    idx = np.argsort([abs(x) for x in temp_polar])[-n:]
    for i in idx:
      print(antonym[i],temp_polar[i],'\n')


  if len(antonym) == len(temp_polar):
    for a in range(len(antonym)):
      temp_dict[antonym[a]] = temp_polar[a]
    return temp_dict




time: 1.59 ms (started: 2022-03-22 15:58:26 +01:00)


In [37]:
jpm_polar = make_polar_dict('jpm', antonym_500, ticker_new_embedding, True)

('impossible', 'probable') -0.10476119 

('distribution', 'hoarding') -0.10681351 

('hail', 'snow') -0.10899254 

('loathe', 'pine') 0.10918275 

('kill', 'reanimate') 0.11325264 

('sidewalk', 'street') -0.11691613 

('school', 'work') -0.12668113 

('apple', 'tomato') 0.13062127 

('practice', 'theory') 0.13276422 

('we', 'you') 0.14179388 

time: 5.95 ms (started: 2022-03-22 15:58:27 +01:00)


In [38]:
amzn_polar = make_polar_dict('amzn', antonym_500, ticker_new_embedding, True)

('lop', 'secure') -0.100378804 

('mate', 'split') -0.10372001 

('cell', 'freedom') -0.109427296 

('handle', 'spout') -0.11193576 

('breathless', 'pulmonary') 0.1210895 

('dash', 'dot') -0.122882806 

('relax', 'toil') -0.12576643 

('impossible', 'probable') -0.13877329 

('loathe', 'pine') 0.15112714 

('apple', 'tomato') 0.20126188 

time: 18.8 ms (started: 2022-03-22 17:13:02 +01:00)


In [ ]:
wmt_polar = make_polar_dict('wmt', antonym_500, ticker_new_embedding, True)

In [ ]:
fb_polar = make_polar_dict('fb', antonym_500, ticker_new_embedding, True)

In [ ]:
wfc_polar = make_polar_dict('wfc', antonym_500, ticker_new_embedding, True) # wells fargo